### Libraries

In [1]:
import importlib

import config as Config
import constants.labels as Labels
import hangar.FirmBomberFactory as FirmBomberFactory
import hangar.FundBomberFactory as FundBomberFactory
import hangar.DisplayModule as DisplayModule
import models.firm as FirmModel
import models.funding as FundingModel
import models.investor as InvestorModel
import models.macro as MacroModel
import utils.dataframe as dfUtils
import utils.file as FileUtils
import utils.industry as IndustryUtils
import utils.visualiser as Visualiser
import utils.tests as Tests

def reload():
  importlib.reload(Config)
  importlib.reload(Visualiser)
  importlib.reload(DisplayModule)
  importlib.reload(Tests)
  importlib.reload(FirmBomberFactory)
  importlib.reload(FundBomberFactory)

### Configuration
Configuration specified in src/config.js to avoid passing the same configurations around. Modify configurations as required, then run reload() to reload configurations without restarting the environment.

### Load Data
Data obtained from Crunchbase and CEIC (for macro data), preprocessed, then stored in pickle files. This section retrieves the stored data and does additional preprocessing.

#### Load Investor Data

In [2]:
# Read Investor Data
investors = FileUtils.read_pickle(f"investors")

In [3]:
# Get Public Funded Investors
public_funded_investors = InvestorModel.get_public_funded_investors(investors)

#### Load Company Data

In [4]:
# Read Company Data
firms = FileUtils.read_pickle(f"{Config.country}_firms")

In [5]:
# Read Domain Data
domain_created_year = FileUtils.read_pickle('domain_created_year')

In [6]:
# Filter and Enrich Company Data
firms = FirmModel.filter_for_profit(firms)
firms = FirmModel.enrich_founded_year(firms, domain_created_year)
firms = FirmModel.filter_founded_year(firms, Config.start_year, Config.end_year)
firms = FirmModel.enrich_public_funded(firms, public_funded_investors)
firms = firms.reset_index(drop=True)

#### Load Funding Data

In [7]:
# Read Funding Data
funding = FileUtils.read_pickle(f"funding")

In [8]:
# Filter and Enrich Funding Data
funding = FundingModel.filter_announced_year(funding, Config.start_year, Config.end_year)
funding = FundingModel.enrich_public_funded(funding, public_funded_investors)

#### Load Macro Data

In [9]:
# Read Macro Data
real_gdp = FileUtils.read_pickle('real_gdp')
fed_rate = FileUtils.read_pickle('fed_rate')

In [10]:
# Filter Macro Data
real_gdp = real_gdp['United States']
real_gdp = real_gdp[real_gdp.index >= Config.start_year]
real_gdp = real_gdp[real_gdp.index < Config.end_year]

fed_rate = fed_rate['Fed Rate']
fed_rate = fed_rate[fed_rate.index >= Config.start_year]
fed_rate = fed_rate[fed_rate.index < Config.end_year]

In [11]:
# Store Macro Data
macro = MacroModel.Macro(real_gdp, fed_rate)

### Load Other Data

In [12]:
# Industry Labels
industry_groups = IndustryUtils.get_industry_groups()
industries = IndustryUtils.get_industries()

### Preprocess Data

In [13]:
# Load Bombers
# reload()
firm_bomber_i = FirmBomberFactory.FirmBomber(firms, targets=industries, target_col=Labels.industries)
firm_bomber_ig = FirmBomberFactory.FirmBomber(firms, targets=industry_groups, target_col=Labels.industry_groups)

In [14]:
# Load Bombers
# reload()
fund_bomber_ig = FundBomberFactory.FundBomber(funding, targets=industry_groups, target_col=Labels.industry_groups)
fund_bomber_i = FundBomberFactory.FundBomber(funding, targets=industries, target_col=Labels.industries)

### Research Question: "Trend-Following"
Have company founders and investors become more prone to trend-following over time, rather than relying on their own judgement? Trend-following results in bubbly behaviour, with sharp spikes and sharp drops, likely cause issues as bad causes are funded in the bubble and good causes are not funded in the crash. Note that a bubble can be defined as "trend-following" or as "deviation from fundamentals"; it's possible to follow a trend and by coincidence approach the fundamentals, though this likely does not hold true in the long run. I will use the "trend-following" definition, since "fundamental" are difficult to define, and understanding trend-following behaviour is still is empirically useful.

I use a year range of 1960 to 2019 as that is what is available on both Crunchbase and CEIC (for macro data), as well as to avoid what looks like incomplete data post 2019. There are dramatically few companies being reported to be founded around that time period, which may be partially attributed to the Coronavirus pandemic, but is still somewhat suspect.

In [15]:
# reload()
# Convert data to data frame for panel regression
# beta_map = fund_bomber_ig.year_count_growth_beta_map
# beta_map = fund_bomber_i.year_count_growth_beta_map
beta_map = firm_bomber_ig.year_count_growth_beta_map
# beta_map = firm_bomber_i.year_count_growth_beta_map

df = dfUtils.map_to_df(beta_map)
df = Tests.prep_panel(df, macro.fed_rate, 'fed_rate')

results = Tests.panel(df, 'fed_rate', entities=True)

/Users/celine/Projects/IO/venv/lib/python3.8/site-packages/linearmodels/panel/model.py:1216: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  super().__init__(dependent, exog, weights=weights, check_rank=check_rank)


results:                           PanelOLS Estimation Summary                           
Dep. Variable:          rolling_value   R-squared:                        0.0503
Estimator:                   PanelOLS   R-squared (Between):              1.0000
No. Observations:                2860   R-squared (Within):            5.804e-05
Date:                Sat, Mar 02 2024   R-squared (Overall):              0.0503
Time:                        19:23:56   Log-likelihood                   -9791.2
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      2.9745
Entities:                          49   P-value                           0.0000
Avg Obs:                       58.367   Distribution:                 F(50,2809)
Min Obs:                       44.000                                           
Max Obs:                       60.000   F-statistic (robust):            8.6e+17
                   